## Word2Vec SkipGram Model Example
This notebook implements a skipgram model.

In [1]:
import tensorflow as tf
import numpy as np

### Model Class

We can use classes to encapsulate tensorflow models. The below skeleton shows one way of using classes with tensorflow.

The Model class has methods, variables, and properties that capture both the graph and the tensorflow session

#### Tensorflow Graph

A tensorflow graph is a computational graph of different tensorflow operations. It defines the computation and how different operations and tensors relate, but it doesn't actually do the computation or store the values of the variables. All of that magic happens within the tensorflow session.

#### Tensorflow Session

A tensorflow session is the context where values for tensorflow variables are instantiated and computations are run. So if you are saving a model's weights, you are actually saving the weights of the tensorflow session. If you are loading a model's weights, you need to load them into a session. When variables are initialized, that has to happen within a session. In a way, the graph is stateless. State is stored in sessions. The session also takes care of running computations, so if you are running training, those need to be run in the session.

A session is instantiated with a graph, typically the current default graph. A session is only able to run computations on the graph that is tied to the session.

In [2]:
class DeepLearningModel():
    def __init__():
        return
    
    def gen_uniform_random_weights(self, k_out, k_in, scale, dtype=np.float32):
        """
        Returns weights of shape (k_in, k_out) initialized between [-scale, scale]
        """
        return ((np.random.rand(k_in, k_out) * 2 - 1) * scale).astype(dtype)

    def gen_random_weights_tanh(self, k_out, k_in, dtype=np.float32):
        scale = (6. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_random_weights_sigmoid(self, k_out, k_in, dtype=np.float32):
        scale = 4. * (6. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_random_weights_reLu(self, k_out, k_in, dtype=np.float32):
        scale = (2. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_biases(self, k, dtype=np.float32):
        """
        Initialize biases as zero.
        """
        return np.zeros((k, ), dtype=dtype)
    
    def clip_gradient(self, grad, magnitude=1.0):
        """returns a clipped gradient, where it is between [-magnitude and magnitude]"""
        magnitude = abs(magnitude)
        return tf.maximum(tf.minimum(grad, magnitude), - magnitude)

---
There are several options for cost

- Negative Sampling
- NCE Loss
- Sampled Soft Max

In [192]:
class SkipGram(DeepLearningModel):
    """
    Tutorial Model
    """
    
    def __init__(self, k_embedding, n_embeddings, n_negative_sample):
        """
        args:
            num_layers: number of hidden layers
            k_hidden: number of units in the hidden layers
            k_embedding: dimensionality of the input
            k_softmax: dimensionality of the output layer
        """
        self._graph = None
        self._session = None
        self.k_embedding = k_embedding
        self.n_embeddings = n_embeddings
        self.n_negative_sample = n_negative_sample
        
        self._merged_training_summary = None
        self._merged_validation_summary = None
    
    
    def load_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
        
        self._session = tf.Session(graph=self.graph)
        model_saver.restore(self._session, model_filename)
        return
    
    
    def save_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
            
        model_saver.save(self.session, model_filename)
        
    def create_graph(self):
        self.cells = {}
                        
        self._graph = tf.Graph()
        with self._graph.as_default():
            with tf.name_scope("inputs"):
                self.X_center_word = tf.placeholder(tf.int32, shape=(None), name="X_center_word")
                self.X_outer_word = tf.placeholder(tf.int32, shape=(None), name="X_outer_word")
                self.X_negative_sample = tf.placeholder(tf.int32, shape=(None, self.n_negative_sample), name="X_outer_word")
                self.learning_rate = tf.placeholder(tf.float32, shape=())
            
            with tf.name_scope("embeddings"):
                self.center_word_embeddings = tf.Variable(np.random.rand(self.n_embeddings, self.k_embedding).astype(np.float32), dtype=tf.float32, name="center_word_embeddings")
                self.outer_word_embeddings = tf.Variable(np.random.rand(self.n_embeddings, self.k_embedding).astype(np.float32), dtype=tf.float32, name="outer_word_embeddings")
                self.outer_word_biases = tf.Variable(np.zeros(self.n_embeddings, dtype=np.float32), name="outer_word_biases")
                
                self.center_word_embedding_lookup = tf.nn.embedding_lookup(self.center_word_embeddings, self.X_center_word)
                self.center_word_embedding_lookup_expanded = tf.expand_dims(tf.nn.embedding_lookup(self.center_word_embeddings, self.X_center_word), axis = 1)
                self.outer_word_embedding_lookup = tf.nn.embedding_lookup(self.outer_word_embeddings, self.X_outer_word)
                self.outer_word_bias_lookup = tf.nn.embedding_lookup(self.outer_word_biases, self.X_outer_word)
                self.negative_sample_embedding_lookup = tf.nn.embedding_lookup(self.outer_word_embeddings, self.X_negative_sample)
                self.negative_sample_bias_lookup = tf.nn.embedding_lookup(self.outer_word_biases, self.X_negative_sample)
                
            
            with tf.name_scope("loss"):
                self.objective_target = tf.sigmoid(tf.reduce_sum(tf.multiply(self.center_word_embedding_lookup, self.outer_word_embedding_lookup), axis = 1) + self.outer_word_bias_lookup)
                self.test = tf.reduce_sum(tf.multiply(- self.center_word_embedding_lookup_expanded, self.negative_sample_embedding_lookup), axis=2)
                self.objective_negative_sample = tf.reduce_sum(tf.sigmoid(tf.reduce_sum(tf.multiply(- self.center_word_embedding_lookup_expanded, self.negative_sample_embedding_lookup), axis=2) + self.negative_sample_bias_lookup), axis=1)
                self.loss = tf.reduce_mean(- (self.objective_target + self.objective_negative_sample))
            
            
            with tf.name_scope("optimization"):
                self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate)
                self.grads_and_vars = self.optimizer.compute_gradients(self.loss)                    
                self.clipped_grads_and_vars = [(self.clip_gradient(gv[0]), gv[1]) for gv in self.grads_and_vars]
                self.update_op = self.optimizer.apply_gradients(self.clipped_grads_and_vars)

            self.init_op = tf.global_variables_initializer()
        return
            

    def write_graph(self):
        self.tensorboard_writer.add_graph(self.graph)
        return
    
    def init_model(self, adam_beta1=0.9, adam_beta2=0.999):
        self.session.run(self.init_op)
    
    @property
    def graph(self):
        if self._graph is None:
            self.create_graph()
        return self._graph
    
    @property
    def session(self):
        if self._session is None:
            self._session = tf.Session(graph=self.graph)
        return self._session

In [193]:
k_embedding=2
n_embeddings=100
n_negative_sample= 8

learning_rate = .1
minibatch_size = 3

In [194]:
# create a model instance with 2 hidden layers and 10 hidden units.

model = SkipGram(k_embedding, n_embeddings, n_negative_sample)

In [195]:
model.graph

In [196]:
model.init_model()

In [197]:
center_word = np.random.randint(0, n_embeddings, size=(minibatch_size, ))
target_word = np.random.randint(0, n_embeddings, size=(minibatch_size, ))
negative_samples = np.random.randint(0, n_embeddings, size=(minibatch_size, n_negative_sample))

In [198]:
model.session.run(model.center_word_embedding_lookup, feed_dict={model.X_center_word: center_word})

array([[ 0.02730493,  0.48873454],
       [ 0.09974992,  0.55979162],
       [ 0.40553328,  0.76625854]], dtype=float32)

In [199]:
model.session.run(model.outer_word_embedding_lookup, feed_dict={model.X_outer_word: target_word})

array([[ 0.68758434,  0.68085098],
       [ 0.19688462,  0.55540323],
       [ 0.86161876,  0.52179176]], dtype=float32)

In [200]:
model.session.run(model.negative_sample_embedding_lookup, feed_dict={model.X_negative_sample: negative_samples}).shape

(3, 8, 2)

In [201]:
model.session.run(model.negative_sample_bias_lookup, feed_dict={model.X_negative_sample: negative_samples})

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [202]:
model.session.run(model.objective_target, feed_dict={model.X_center_word: center_word, 
                                                     model.X_outer_word: target_word,
                                                     model.X_negative_sample: negative_samples})

array([ 0.58698851,  0.58189303,  0.67901361], dtype=float32)

In [203]:
model.session.run(model.objective_negative_sample, feed_dict={model.X_center_word: center_word, 
                                                              model.X_outer_word: target_word,
                                                              model.X_negative_sample: negative_samples})

array([ 3.60950661,  3.60912609,  2.98638296], dtype=float32)

In [204]:
model.session.run(model.loss, feed_dict={model.X_center_word: center_word, 
                                         model.X_outer_word: target_word,
                                         model.X_negative_sample: negative_samples})

-4.0176368

In [205]:
model.session.run(model.test, feed_dict={model.X_center_word: center_word, 
                                         model.X_outer_word: target_word,
                                         model.X_negative_sample: negative_samples})

array([[-0.0108158 , -0.04481015, -0.18595314, -0.26334992, -0.2777091 ,
        -0.24344674, -0.17242643, -0.37317902],
       [-0.2641167 , -0.40357202, -0.33077899, -0.0796399 , -0.13241777,
        -0.0303914 , -0.065114  , -0.26923794],
       [-0.65398818, -0.38898832, -0.37631741, -0.83073562, -0.37631741,
        -0.74791992, -0.4793309 , -0.3263647 ]], dtype=float32)

In [207]:
model.session.run(model.update_op, feed_dict={model.X_center_word: center_word, 
                                              model.X_outer_word: target_word,
                                              model.X_negative_sample: negative_samples,
                                              model.learning_rate: learning_rate})

In [208]:
model.session.run(model.test, feed_dict={model.X_center_word: center_word, 
                                         model.X_outer_word: target_word,
                                         model.X_negative_sample: negative_samples})

array([[ -4.15414048e-04,  -3.49701159e-02,  -1.76525638e-01,
         -2.40010083e-01,  -2.55895585e-01,  -2.20351174e-01,
         -1.48479238e-01,  -3.37162435e-01],
       [ -2.54901946e-01,  -3.63368511e-01,  -3.02406490e-01,
         -6.83926493e-02,  -9.87522975e-02,  -1.72795914e-02,
         -4.26510386e-02,  -2.47127593e-01],
       [ -6.22921228e-01,  -3.63044143e-01,  -3.39324653e-01,
         -7.97489941e-01,  -3.39324653e-01,  -7.14577079e-01,
         -4.60051268e-01,  -3.00982594e-01]], dtype=float32)

In [209]:
model.session.run(model.loss, feed_dict={model.X_center_word: center_word, 
                                         model.X_outer_word: target_word,
                                         model.X_negative_sample: negative_samples})

-4.0819592

In [211]:
model.session.run(model.update_op, feed_dict={model.X_center_word: center_word, 
                                              model.X_outer_word: target_word,
                                              model.X_negative_sample: negative_samples,
                                              model.learning_rate: learning_rate})

In [212]:
model.session.run(model.test, feed_dict={model.X_center_word: center_word, 
                                         model.X_outer_word: target_word,
                                         model.X_negative_sample: negative_samples})

array([[ 0.00951132, -0.02536188, -0.16742112, -0.21698518, -0.23441185,
        -0.19755943, -0.12478894, -0.30147463],
       [-0.24609254, -0.32355672, -0.27441859, -0.05740866, -0.06616244,
        -0.00460211, -0.02040374, -0.22538079],
       [-0.59256208, -0.33774155, -0.30386877, -0.76500458, -0.30386877,
        -0.68195629, -0.44153962, -0.27621585]], dtype=float32)

In [213]:
model.session.run(model.loss, feed_dict={model.X_center_word: center_word, 
                                         model.X_outer_word: target_word,
                                         model.X_negative_sample: negative_samples})

-4.145658

Woot! It's learning!

---